### Define and Execute dNBR Calculation
Next we define a function that will interatively query for Sentinel-2 NBART imagery, calculate a dNBR tile and write the output to a local GeoTIFF file store in a subfolder. Once defined, we iterate through all the coordinates to compute all the dNBR tiles.

> **Note:** Using the default notebook parameters on the NCI VDI, this will take approximately 30mins. The NCI VDI allocates users with a 2GB local storage allocation. If you exceed this limit, try reducing your spatial extent or temporal range.

In [ ]:
def dNBR_processing(coordinates):

    # Load all data in baseline period available from s2a/b_ard_granule datasets
    prefire_ard = load_ard(
        dc=dc,
        products=["s2a_ard_granule", "s2b_ard_granule"],
        x=(coordinates.x - 0.1, coordinates.x + 0.1),
        y=(coordinates.y - 0.1, coordinates.y + 0.1),
        time=(prefire_start, prefire_end),
        measurements=["nbart_nir_1", "nbart_swir_3"],
        min_gooddata=0.1,
        output_crs="EPSG:32755",  # UTM Zone 55S
        resolution=(-10, 10),
        group_by="solar_day",
    )

    prefire_ard = calculate_indices(
        prefire_ard, index="NBR", collection="ga_s2_1", drop=False
    )

    # Compute median using all observations in the dataset along the time axis
    prefire_image = prefire_ard.median(dim="time")

    # Delete baseline_combined
    del prefire_ard

    # Select NBR
    prefire_NBR = prefire_image.NBR

    del prefire_image

    # Load all data in post-fire period available from s2a/b_ard_granule datasets
    postfire_ard = load_ard(
        dc=dc,
        products=["s2a_ard_granule", "s2b_ard_granule"],
        x=(coordinates.x - 0.1, coordinates.x + 0.1),
        y=(coordinates.y - 0.1, coordinates.y + 0.1),
        time=(postfire_start, postfire_end),
        measurements=["nbart_nir_1", "nbart_swir_3"],
        min_gooddata=0.1,
        output_crs="EPSG:32755",  # UTM Zone 55S
        resolution=(-10, 10),
        group_by="solar_day",
    )

    # Calculate NBR on all post-fire images
    postfire_ard = calculate_indices(
        postfire_ard, index="NBR", collection="ga_s2_1", drop=False
    )

    # Calculate the median post-fire image
    postfire_image = postfire_ard.median(dim="time")

    del postfire_ard

    # Select NBR
    postfire_NBR = postfire_image.NBR

    del postfire_image

    # Calculate delta
    delta_NBR = prefire_NBR - postfire_NBR

    del prefire_NBR
    del postfire_NBR

    x = np.round_(coordinates.x, decimals=4)
    y = np.round_(coordinates.y, decimals=4)

    # Turn dNBR into a x-array dataset for export to GeoTIFF
    dnbr_dataset = delta_NBR.to_dataset(name="delta_NBR")
    write_geotiff(f"./dNBR_geotiffs/{x}_{y}_dNBR.tif", dnbr_dataset)

    del delta_NBR
    del dnbr_dataset

In [ ]:
%%time

# Iterate through all shapely points to generate a dNBR geotiff.
for i in coords:
    dNBR_processing(i)